# Import dependencies

In [ ]:
import os
import sys

sys.path.insert(0, os.path.dirname(os.getcwd())) 

In [1]:
import time
import gc

import numpy as np
import pandas as pd

from transformers import BertTokenizer, BertModel

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

from semeval_reader import SemevalReader

from InputDataset import InputDataset

import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from torch import cuda

from models.BERT_Dropout_BiLSTM_Linear import BERT_Dropout_BiLSTM_Linear

In [2]:
device = 'cuda' if cuda.is_available() else 'cpu'

In [3]:
print(torch.cuda.get_device_name(0))
print(f"Memory: {torch.cuda.get_device_properties(0).total_memory // 1024 ** 3} GB")

NVIDIA GeForce RTX 2060 SUPER
Memory: 8 GB


In [4]:
def clear_memory():
    torch.cuda.empty_cache()

    with torch.no_grad():
        torch.cuda.empty_cache()

    gc.collect()

# Load Data

In [5]:
def get_target_list_for_polarity(polarity):
    if polarity == 'positive':
        return [0, 0, 1]
    if polarity == 'negative':
        return [1, 0, 0]
    return [0, 1, 0]

In [6]:
semeval_reader = SemevalReader('../../../data/semeval16_restaurants_train.xml')

reviews = semeval_reader.read_reviews()
absolute_polarity_sentences = semeval_reader.get_absolute_polarity_sentences()

df = pd.DataFrame(map(lambda x: (x.text, x.opinions[0].polarity), absolute_polarity_sentences))
df.rename(columns={0: 'text'}, inplace=True)
df['target_list'] = df.apply(lambda row: get_target_list_for_polarity(row[1]), axis=1)

absolute_polarity_df = df.drop(columns=[1])

# Train & Validate

In [7]:
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4

EPOCHS = 2

LEARNING_RATE = 1e-5

TRAIN_SPLIT = 0.8

NO_RUNS = 10

In [8]:
BERT_FINE_TUNED_PATH = '../../../results/SA/SemEval16 - Task 5 - Restaurants/models/bert_fine_tuned.pth'

In [9]:
MODEL_OUTPUT = '../../../results/SA/SemEval16 - Task 5 - Restaurants/models/bert_fine_tuned_dropout_bilstm_linear.pth'
STATS_OUTPUT = '../../../results/SA/SemEval16 - Task 5 - Restaurants/stats/bert_fine_tuned_dropout_bilstm_linear.csv'

In [10]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [11]:
best_accuracy = 0.0

In [12]:
def train(epoch, model, loss_fn, optimizer, dataloader):
    model.train()

    dataloader_len = len(dataloader)

    for _,data in enumerate(dataloader, 0):
        optimizer.zero_grad()

        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        loss = loss_fn(outputs, targets)
        
        if _ % (dataloader_len // 10) == 0:
            print(f"Epoch: {epoch}/{EPOCHS}, Batch: {_}/{dataloader_len}, Loss: {loss.item()}")
        
        loss.backward()
        
        optimizer.step()

In [13]:
def validation(model, dataloader):
    model.eval()
    
    fin_targets=[]
    fin_outputs=[]

    with torch.no_grad():
        for _, data in enumerate(dataloader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)

            outputs = model(ids, mask, token_type_ids)
            
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

    return fin_outputs, fin_targets

In [14]:
results = pd.DataFrame(columns=['accuracy','precision_score_micro','precision_score_macro','recall_score_micro','recall_score_macro','f1_score_micro','f1_score_macro', 'execution_time'])

In [15]:
for i in range(10):
    # clear cache cuda
    torch.cuda.empty_cache()
    with torch.no_grad():
        torch.cuda.empty_cache()
    gc.collect()

    start_time = time.time()

    print(f"Run {i + 1}/10")

    train_dataset = df.sample(frac=TRAIN_SPLIT)
    test_dataset = df.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    training_set = InputDataset(train_dataset, tokenizer)
    testing_set = InputDataset(test_dataset, tokenizer)

    train_dataloader = DataLoader(
        training_set,
        sampler = RandomSampler(train_dataset),
        batch_size = TRAIN_BATCH_SIZE,
        drop_last = True
    )

    validation_dataloader = DataLoader(
        testing_set,
        sampler = SequentialSampler(testing_set),
        batch_size = VALID_BATCH_SIZE,
        drop_last = True
    )

    model = BERT_Dropout_BiLSTM_Linear(torch.load(BERT_FINE_TUNED_PATH), dropout=0.3, bilstm_in_features=256, no_out_labels=3, device=device).to(device)

    optimizer = torch.optim.Adam(params = model.parameters(), lr=LEARNING_RATE)
    loss_fn = torch.nn.BCEWithLogitsLoss()

    for epoch in range(EPOCHS):
        train(epoch, model, loss_fn, optimizer, train_dataloader)

    outputs, targets = validation(model, validation_dataloader)
    outputs = np.argmax(outputs, axis=1)
    targets = np.argmax(targets, axis=1)
    
    accuracy = accuracy_score(targets, outputs)
    precision_score_micro = precision_score(targets, outputs, average='micro')
    precision_score_macro = precision_score(targets, outputs, average='macro')
    recall_score_micro = recall_score(targets, outputs, average='micro')
    recall_score_macro = recall_score(targets, outputs, average='macro')
    f1_score_micro = f1_score(targets, outputs, average='micro')
    f1_score_macro = f1_score(targets, outputs, average='macro')

    execution_time = time.time() - start_time

    results.loc[i] = [accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro, execution_time]

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model.bert, MODEL_OUTPUT)

    del train_dataset
    del test_dataset
    del training_set
    del testing_set
    del model
    del loss_fn
    del optimizer
    del outputs
    del targets

Run 1/10
Epoch: 0/2, Batch: 0/313, Loss: 0.6921728849411011
Epoch: 0/2, Batch: 31/313, Loss: 0.596758246421814
Epoch: 0/2, Batch: 62/313, Loss: 0.5623652935028076
Epoch: 0/2, Batch: 93/313, Loss: 0.4574130177497864
Epoch: 0/2, Batch: 124/313, Loss: 0.48909252882003784
Epoch: 0/2, Batch: 155/313, Loss: 0.3319976329803467
Epoch: 0/2, Batch: 186/313, Loss: 0.35142624378204346
Epoch: 0/2, Batch: 217/313, Loss: 0.3214929699897766
Epoch: 0/2, Batch: 248/313, Loss: 0.26520201563835144
Epoch: 0/2, Batch: 279/313, Loss: 0.4001624584197998
Epoch: 0/2, Batch: 310/313, Loss: 0.24044887721538544
Epoch: 1/2, Batch: 0/313, Loss: 0.23136810958385468
Epoch: 1/2, Batch: 31/313, Loss: 0.178672194480896
Epoch: 1/2, Batch: 62/313, Loss: 0.21965515613555908
Epoch: 1/2, Batch: 93/313, Loss: 0.20189473032951355
Epoch: 1/2, Batch: 124/313, Loss: 0.1535121351480484
Epoch: 1/2, Batch: 155/313, Loss: 0.15459716320037842
Epoch: 1/2, Batch: 186/313, Loss: 0.14692473411560059
Epoch: 1/2, Batch: 217/313, Loss: 0.2097

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 2/10
Epoch: 0/2, Batch: 0/313, Loss: 0.7350912094116211
Epoch: 0/2, Batch: 31/313, Loss: 0.5977069735527039
Epoch: 0/2, Batch: 62/313, Loss: 0.5859463214874268
Epoch: 0/2, Batch: 93/313, Loss: 0.49579280614852905
Epoch: 0/2, Batch: 124/313, Loss: 0.43000268936157227
Epoch: 0/2, Batch: 155/313, Loss: 0.63702392578125
Epoch: 0/2, Batch: 186/313, Loss: 0.38562750816345215
Epoch: 0/2, Batch: 217/313, Loss: 0.4453274607658386
Epoch: 0/2, Batch: 248/313, Loss: 0.28522995114326477
Epoch: 0/2, Batch: 279/313, Loss: 0.29288697242736816
Epoch: 0/2, Batch: 310/313, Loss: 0.2503259778022766
Epoch: 1/2, Batch: 0/313, Loss: 0.24270980060100555
Epoch: 1/2, Batch: 31/313, Loss: 0.18171286582946777
Epoch: 1/2, Batch: 62/313, Loss: 0.16898857057094574
Epoch: 1/2, Batch: 93/313, Loss: 0.16085851192474365
Epoch: 1/2, Batch: 124/313, Loss: 0.39402318000793457
Epoch: 1/2, Batch: 155/313, Loss: 0.17243170738220215
Epoch: 1/2, Batch: 186/313, Loss: 0.1596912294626236
Epoch: 1/2, Batch: 217/313, Loss: 0.17

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 3/10
Epoch: 0/2, Batch: 0/313, Loss: 0.6462710499763489
Epoch: 0/2, Batch: 31/313, Loss: 0.5706930160522461
Epoch: 0/2, Batch: 62/313, Loss: 0.5183661580085754
Epoch: 0/2, Batch: 93/313, Loss: 0.6283471584320068
Epoch: 0/2, Batch: 124/313, Loss: 0.3882758617401123
Epoch: 0/2, Batch: 155/313, Loss: 0.46942758560180664
Epoch: 0/2, Batch: 186/313, Loss: 0.32941821217536926
Epoch: 0/2, Batch: 217/313, Loss: 0.26383742690086365
Epoch: 0/2, Batch: 248/313, Loss: 0.3904573619365692
Epoch: 0/2, Batch: 279/313, Loss: 0.2506711781024933
Epoch: 0/2, Batch: 310/313, Loss: 0.3553710877895355
Epoch: 1/2, Batch: 0/313, Loss: 0.19324541091918945
Epoch: 1/2, Batch: 31/313, Loss: 0.24055203795433044
Epoch: 1/2, Batch: 62/313, Loss: 0.17322400212287903
Epoch: 1/2, Batch: 93/313, Loss: 0.18004564940929413
Epoch: 1/2, Batch: 124/313, Loss: 0.1517874300479889
Epoch: 1/2, Batch: 155/313, Loss: 0.14208823442459106
Epoch: 1/2, Batch: 186/313, Loss: 0.12841634452342987
Epoch: 1/2, Batch: 217/313, Loss: 0.09

...

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 9/10
Epoch: 0/2, Batch: 0/313, Loss: 0.6917068362236023
Epoch: 0/2, Batch: 31/313, Loss: 0.6001849174499512
Epoch: 0/2, Batch: 62/313, Loss: 0.4963928461074829
Epoch: 0/2, Batch: 93/313, Loss: 0.4540519118309021
Epoch: 0/2, Batch: 124/313, Loss: 0.38054391741752625
Epoch: 0/2, Batch: 155/313, Loss: 0.3440180718898773
Epoch: 0/2, Batch: 186/313, Loss: 0.3250044584274292
Epoch: 0/2, Batch: 217/313, Loss: 0.2661232054233551
Epoch: 0/2, Batch: 248/313, Loss: 0.23420056700706482
Epoch: 0/2, Batch: 279/313, Loss: 0.21900153160095215
Epoch: 0/2, Batch: 310/313, Loss: 0.6726834177970886
Epoch: 1/2, Batch: 0/313, Loss: 0.18872790038585663
Epoch: 1/2, Batch: 31/313, Loss: 0.1763370931148529
Epoch: 1/2, Batch: 62/313, Loss: 0.2165035456418991
Epoch: 1/2, Batch: 93/313, Loss: 0.1952580213546753
Epoch: 1/2, Batch: 124/313, Loss: 0.16558456420898438
Epoch: 1/2, Batch: 155/313, Loss: 0.1970665156841278
Epoch: 1/2, Batch: 186/313, Loss: 0.14578746259212494
Epoch: 1/2, Batch: 217/313, Loss: 0.13572

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 10/10
Epoch: 0/2, Batch: 0/313, Loss: 0.7071658372879028
Epoch: 0/2, Batch: 31/313, Loss: 0.576979398727417
Epoch: 0/2, Batch: 62/313, Loss: 0.5069988965988159
Epoch: 0/2, Batch: 93/313, Loss: 0.40520983934402466
Epoch: 0/2, Batch: 124/313, Loss: 0.38857099413871765
Epoch: 0/2, Batch: 155/313, Loss: 0.6082031726837158
Epoch: 0/2, Batch: 186/313, Loss: 0.33999618887901306
Epoch: 0/2, Batch: 217/313, Loss: 0.31050917506217957
Epoch: 0/2, Batch: 248/313, Loss: 0.24409881234169006
Epoch: 0/2, Batch: 279/313, Loss: 0.2218974083662033
Epoch: 0/2, Batch: 310/313, Loss: 0.23529872298240662
Epoch: 1/2, Batch: 0/313, Loss: 0.20530512928962708
Epoch: 1/2, Batch: 31/313, Loss: 0.1768535077571869
Epoch: 1/2, Batch: 62/313, Loss: 0.16780202090740204
Epoch: 1/2, Batch: 93/313, Loss: 0.18106552958488464
Epoch: 1/2, Batch: 124/313, Loss: 0.11753490567207336
Epoch: 1/2, Batch: 155/313, Loss: 0.13258586823940277
Epoch: 1/2, Batch: 186/313, Loss: 0.15021422505378723
Epoch: 1/2, Batch: 217/313, Loss: 0

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
results

,accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro,execution_time
0,0.942308,0.942308,0.623571,0.942308,0.651796,0.942308,0.637233,223.821147
1,0.923077,0.923077,0.593623,0.923077,0.648749,0.923077,0.618035,203.204230
2,0.932692,0.932692,0.600769,0.932692,0.644900,0.932692,0.620413,195.122486
3,0.932692,0.932692,0.606912,0.932692,0.647302,0.932692,0.625178,202.400194
4,0.916667,0.916667,0.595605,0.916667,0.637139,0.916667,0.613427,202.712312
5,0.939103,0.939103,0.951861,0.939103,0.673230,0.939103,0.689602,202.624539
6,0.910256,0.910256,0.595590,0.910256,0.634937,0.910256,0.613845,203.594272
7,0.929487,0.929487,0.607421,0.929487,0.636953,0.929487,0.621651,198.490349
8,0.916667,0.916667,0.600169,0.916667,0.624086,0.916667,0.611323,207.753877
9,0.929487,0.929487,0.608430,0.929487,0.638669,0.929487,0.622987,202.736751


In [17]:
results.to_csv(STATS_OUTPUT)